In [47]:
from clickhouse_driver import Client

In [48]:
clickhouse = Client(
    host='127.0.0.1',
    port=19000,
    user='default',
    password='SkyWalker25!'
)

In [49]:
version = client.execute('SELECT * FROM system.numbers LIMIT 5')
print(version)

[(0,), (1,), (2,), (3,), (4,)]


In [50]:
clickhouse.execute("DROP TABLE logs")

[]

Пример логов:
15.74.45.114 - - [2023-01-05 00:00:03 +0300] "DELETE /usr/admin HTTP/1.0" 304 5016 "-" "Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36" 4533
122.239.116.246 - - [2023-01-05 00:00:05 +0300] "GET /usr/login HTTP/1.0" 200 4973 "http://www.gonzalez.info/explore/explorehome.jsp" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36" 3144
139.31.190.36 - - [2023-01-05 00:00:10 +0300] "GET /usr/admin/developer HTTP/1.0" 403 5087 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36" 4437
196.241.201.130 - - [2023-01-05 00:00:12 +0300] "PUT /usr/admin/developer HTTP/1.0" 404 4989 "http://www.gonzalez.info/explore/explorehome.jsp" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A" 4445
24.165.196.101 - - [2023-01-05 00:00:12 +0300] "PUT /usr/admin HTTP/1.0" 404 4980 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36" 2039
114.87.92.66 - - [2023-01-05 00:00:13 +0300] "PUT /usr/admin/developer HTTP/1.0" 500 4980 "http://www.gonzalez.info/explore/explorehome.jsp" "Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36 EdgA/45.12.4.5121" 3635
190.53.230.43 - - [2023-01-05 00:00:14 +0300] "PUT /usr/register HTTP/1.0" 304 5062 "http://www.gonzalez.info/explore/explorehome.jsp" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36 OPR/73.0.3856.329" 1527
192.56.34.224 - - [2023-01-05 00:00:17 +0300] "GET /usr/admin HTTP/1.0" 200 4939 "http://www.gonzalez.info/explore/explorehome.jsp" "Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_9 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Mobile/15E148 Safari/604.1" 2903
216.99.151.252 - - [2023-01-05 00:00:19 +0300] "PUT /usr/admin HTTP/1.0" 502 5026 "http://www.gonzalez.info/explore/explorehome.jsp" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4380.0 Safari/537.36 Edg/89.0.759.0" 118
175.112.158.124 - - [2023-01-05 00:00:19 +0300] "POST /usr HTTP/1.0" 403 5032 "http://www.gonzalez.info/explore/explorehome.jsp" "Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36" 217

Разработать структуру базы данных (модель типа «звезда» или
«снежинка») для хранения и анализа логов. Логи файлов размещаются
в одном каталоге.

Структура логов (IP-адрес, дата/время, метод, путь, статус, user-agent и т.д.) хорошо подходит под одну «факт»-таблицу, где большая часть полей хранится непосредственно в факт-таблице, а некоторые повторяющиеся поля (например, user-agent, реферер, IP, метод) можно вынести в небольшие «измерения» (dimension tables).

In [51]:
clickhouse.execute("""
CREATE TABLE IF NOT EXISTS logs (
    ip String,
    timestamp DateTime,
    method String,
    path String,
    protocol String,
    status UInt16,
    bytes_sent UInt32,
    referrer String,
    user_agent String,
    response_time UInt32
)
ENGINE = MergeTree()
PARTITION BY toYYYYMM(timestamp)
ORDER BY (timestamp, ip)
    """)

[]

In [ ]:
import re
from datetime import datetime
from collections import defaultdict

pattern = re.compile(
    r'^(?P<ip>\S+)\s-\s-\s\[(?P<datetime>[^\]]+)\]\s'
    r'"(?P<method>\S+)\s(?P<path>\S+)\s(?P<protocol>[^"]+)"\s'
    r'(?P<status>\d+)\s'
    r'(?P<bytes>\d+)\s'
    r'"(?P<referrer>[^"]*)"\s'
    r'"(?P<useragent>[^"]*)"\s'
    r'(?P<response_time>\d+)$'
)
def parse_datetime(dt_string):
    dt_string = dt_string.replace("+0300", "").strip()
    return datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S")


#Подготавливаем batch для вставки в logs
batch_size = 1000
batch_data = []

#Собираем информацию IP -> day -> count
stats_by_ip_and_day = defaultdict(lambda: defaultdict(int))
#Считаем отказы сервера (4хх, 5хх)
refusals_by_ip_and_day = defaultdict(lambda: defaultdict(int))



with open('logfile1.log', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        match = pattern.match(line)
        if match:
            gd = match.groupdict()
            ip = gd["ip"]
            dt_str = gd["datetime"]
            method = gd["method"]
            path_ = gd["path"]
            protocol_ = gd["protocol"]
            status = int(gd["status"])
            bytes_ = int(gd["bytes"])
            referrer_ = gd["referrer"]
            useragent_ = gd["useragent"]
            response_time_ = int(gd["response_time"])
            
            # Парсим время
            dt_obj = parse_datetime(dt_str)

            batch_data.append((
                ip,
                dt_obj,
                method,
                path_,
                protocol_,
                status,
                bytes_,
                referrer_,
                useragent_,
                response_time_
            ))

            day_str = dt_obj.strftime('%Y%m%d')

            stats_by_ip_and_day[ip][day_str] +=1

            if 400 <= status < 600:
                refusals_by_ip_and_day[ip][day_str] += 1

            # Когда накопили достаточно строк — отправляем в ClickHouse
            if len(batch_data) >= batch_size:
                clickhouse.execute(
                    "INSERT INTO logs VALUES",
                    batch_data
                )
                batch_data.clear()

# Вставляем «хвост» (если остались записи в batch_data)
if batch_data:
    clickhouse.execute(
        "INSERT INTO logs VALUES",
        batch_data
    )
    batch_data.clear()

print("Загрузка логов завершена!")

Загрузка логов завершена!
